In [1]:
import os
import glob
import numpy as np
import pandas as pd
from functions import metrics, measure_csv_df
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
os.chdir("/Users/pestephan/Documents/GitHub/Machine-Learning-using-Sports-Data/Predicting-PremLeague-Outcomes-With-ML/files/Datasets")

Data Taken from http://www.football-data.co.uk/englandm.php

In [2]:
extension = 'csv'
all_filenames = [i for i in glob.glob('*.{}'.format(extension))]

In [3]:
#combine all files in the list
master_sheet = pd.concat([pd.read_csv(f) for f in all_filenames ])
#export to csv
master_sheet.to_csv( "master_sheet.csv", index=False, encoding='utf-8-sig')

Reduce table to using columns needed for the model

Columns Required:

Date, Time HomeTeam, AwayTeam, FullTimeHG, FullTimeAG, FullTimeRresults, HalfTimeHG, HalfTimeAG, HomeShots, AwayShots, HomeShotsTarger, AwayShotsTarger, HomeCorners, AwayCorners, HomeFouls, AwayFouls, HomeRed, AwayRed

Columns To Consider:

Attendance, Referee

In [4]:
subset_master = master_sheet[["Date", "Time", "HomeTeam", "AwayTeam", "FTHG", "FTAG", "FTR", "HTHG", "HTAG", "HS", "AS", "HST", "AST", "HC", "AC", "HF", "AF", "HR", "AR"]]
subset_master = subset_master.fillna(0)
# 0 = Home Wins, 1 = Draw, 2 = Away Wins
subset_master.FTR.replace(['H', 'D', 'A'], [0, 1, 2], inplace = True)

In [5]:
subset_master.head()

,Date,Time,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HS,AS,HST,AST,HC,AC,HF,AF,HR,AR
0,13/08/11,0,Blackburn,Wolves,1.0,2.0,2,1.0,1.0,16.0,13.0,8.0,4.0,12.0,6.0,14.0,10.0,0.0,0.0
1,13/08/11,0,Fulham,Aston Villa,0.0,0.0,1,0.0,0.0,13.0,7.0,9.0,1.0,2.0,3.0,10.0,18.0,0.0,0.0
2,13/08/11,0,Liverpool,Sunderland,1.0,1.0,1,1.0,0.0,11.0,15.0,4.0,6.0,6.0,3.0,17.0,12.0,0.0,0.0
3,13/08/11,0,Newcastle,Arsenal,0.0,0.0,1,0.0,0.0,6.0,9.0,1.0,4.0,2.0,5.0,9.0,11.0,0.0,1.0
4,13/08/11,0,QPR,Bolton,0.0,4.0,2,0.0,1.0,13.0,13.0,7.0,7.0,3.0,2.0,9.0,16.0,1.0,0.0


In [6]:
msk = np.random.rand(len(subset_master)) < 0.75

train_df = subset_master[msk]
test_df = subset_master[~msk]

X_train = train_df.drop(columns = ['Date', 'Time', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR', "HTHG", "HTAG"])
y_train = train_df[['FTR']] 
X_test = test_df.drop(columns = ['Date', 'Time', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR', "HTHG", "HTAG", ])
y_test = test_df[['FTR']]

In [7]:
clf = RandomForestClassifier(n_estimators = 2000, max_depth=3, random_state=0)
clf.fit(X_train, y_train)

y_pred = clf.predict_proba(X_test)
y_pred = pd.DataFrame(y_pred)

Identifying how often Predicted Results decided the outcome (win, draw, away) against the actual results)

In [8]:
metrics(subset_master, y_pred)

False    58.262844
True     41.737156
Name: Comparison, dtype: float64
0    45.574748
2    30.148696
1    24.276556
Name: FTR, dtype: float64
0    72.978809
2    27.021191
Name: PredResult, dtype: float64


In [9]:
subset_master

,Date,Time,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HS,AS,HST,AST,HC,AC,HF,AF,HR,AR,HomeWinPerc,DrawPerc,AwayPerc
0,13/08/11,0,Blackburn,Wolves,1.0,2.0,2,1.0,1.0,16.0,13.0,8.0,4.0,12.0,6.0,14.0,10.0,0.0,0.0,0.509844,0.238583,0.251573
1,13/08/11,0,Fulham,Aston Villa,0.0,0.0,1,0.0,0.0,13.0,7.0,9.0,1.0,2.0,3.0,10.0,18.0,0.0,0.0,0.655737,0.213033,0.131230
2,13/08/11,0,Liverpool,Sunderland,1.0,1.0,1,1.0,0.0,11.0,15.0,4.0,6.0,6.0,3.0,17.0,12.0,0.0,0.0,0.469419,0.233757,0.296823
3,13/08/11,0,Newcastle,Arsenal,0.0,0.0,1,0.0,0.0,6.0,9.0,1.0,4.0,2.0,5.0,9.0,11.0,0.0,1.0,0.578123,0.212960,0.208917
4,13/08/11,0,QPR,Bolton,0.0,4.0,2,0.0,1.0,13.0,13.0,7.0,7.0,3.0,2.0,9.0,16.0,1.0,0.0,0.558836,0.219944,0.221220
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
376,24/05/15,0,Leicester,QPR,5.0,1.0,0,2.0,0.0,22.0,18.0,7.0,2.0,5.0,6.0,7.0,6.0,0.0,0.0,0.362660,0.287559,0.349781
377,24/05/15,0,Man City,Southampton,2.0,0.0,0,1.0,0.0,15.0,13.0,6.0,4.0,8.0,4.0,13.0,8.0,0.0,0.0,0.713004,0.185811,0.101185
378,24/05/15,0,Newcastle,West Ham,2.0,0.0,0,0.0,0.0,17.0,4.0,4.0,1.0,2.0,3.0,9.0,9.0,0.0,0.0,0.234429,0.242422,0.523149
379,24/05/15,0,Stoke,Liverpool,6.0,1.0,0,5.0,0.0,15.0,13.0,9.0,4.0,3.0,9.0,13.0,4.0,0.0,0.0,0.605770,0.240989,0.153242


Columns Required for predicting:

HomeShots, AwayShots, HomeShotsTarger, AwayShotsTarger, HomeCorners, AwayCorners, HomeFouls, AwayFouls, HomeRed, AwayRed

In [11]:
current_season = pd.read_csv("/Users/pestephan/Documents/GitHub/Machine-Learning-using-Sports-Data/Predicting-PremLeague-Outcomes-With-ML/2020-2021.csv")

In [12]:
measure_csv_df(current_season, clf)

,Date,Time,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HS,AS,HST,AST,HC,AC,HF,AF,HR,AR,HomeWinPerc,DrawPerc,AwayPerc
0,12/09/2020,12:30,Fulham,Arsenal,0,3,A,0,1,5,13,2,6,2,3,12,12,0,0,0.210958,0.240478,0.548564
1,12/09/2020,15:00,Crystal Palace,Southampton,1,0,H,1,0,5,9,3,5,7,3,14,11,0,0,0.305771,0.269668,0.424560
2,12/09/2020,17:30,Liverpool,Leeds,4,3,H,3,2,22,6,6,3,9,0,9,6,0,0,0.583436,0.228945,0.187619
3,12/09/2020,20:00,West Ham,Newcastle,0,2,A,0,0,15,15,3,2,8,7,13,7,0,0,0.439780,0.289771,0.270449
4,13/09/2020,14:00,West Brom,Leicester,0,3,A,0,0,7,13,1,7,2,5,12,9,0,0,0.221845,0.234098,0.544058
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
220,07/02/2021,12:00,Tottenham,West Brom,2,0,H,0,0,13,4,6,1,8,0,16,12,0,0,0.637466,0.229228,0.133306
221,07/02/2021,14:00,Wolves,Leicester,0,0,D,0,0,13,13,1,3,8,2,11,15,0,0,0.341149,0.304392,0.354459
222,07/02/2021,16:30,Liverpool,Man City,1,4,A,0,0,8,8,3,5,6,1,13,8,0,0,0.316702,0.259718,0.423580
223,07/02/2021,19:15,Sheffield United,Chelsea,1,2,A,0,1,8,9,3,3,1,8,8,8,0,0,0.416776,0.305285,0.277940


In [13]:
#metrics(current_season, y_pred) -- Update y_pred variable with new one for 2021 season

,Date,Time,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HS,...,AST,HC,AC,HF,AF,HR,AR,HomeWinPerc,DrawPerc,AwayPerc
0,12/09/2020,12:30,Fulham,Arsenal,0,3,A,0,1,5,...,6,2,3,12,12,0,0,0.209975,0.240520,0.549505
1,12/09/2020,15:00,Crystal Palace,Southampton,1,0,H,1,0,5,...,5,7,3,14,11,0,0,0.308202,0.269979,0.421819
2,12/09/2020,17:30,Liverpool,Leeds,4,3,H,3,2,22,...,3,9,0,9,6,0,0,0.589815,0.227012,0.183173
3,12/09/2020,20:00,West Ham,Newcastle,0,2,A,0,0,15,...,2,8,7,13,7,0,0,0.440410,0.287641,0.271949
4,13/09/2020,14:00,West Brom,Leicester,0,3,A,0,0,7,...,7,2,5,12,9,0,0,0.220927,0.233733,0.545340
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
220,07/02/2021,12:00,Tottenham,West Brom,2,0,H,0,0,13,...,1,8,0,16,12,0,0,0.639213,0.228904,0.131883
221,07/02/2021,14:00,Wolves,Leicester,0,0,D,0,0,13,...,3,8,2,11,15,0,0,0.339995,0.303626,0.356379
222,07/02/2021,16:30,Liverpool,Man City,1,4,A,0,0,8,...,5,6,1,13,8,0,0,0.321171,0.261877,0.416953
223,07/02/2021,19:15,Sheffield United,Chelsea,1,2,A,0,1,8,...,3,1,8,8,8,0,0,0.417112,0.305421,0.277467
